In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco

In [2]:
# https://data.nber.org/data-appendix/w15319/
# https://data.nber.org/data-appendix/w15319/FinalCHAT_72909.csv
# https://www.nber.org/system/files/working_papers/w15319/w15319.pdf


In [3]:
os.chdir('/Users/ariana/desktop/historical_tech')

In [4]:
chat = pd.read_csv('raw data/FinalCHAT_72909.csv')


/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/3727645712.py:1: DtypeWarning: Columns (46,51,55,56,60) have mixed types. Specify dtype option on import or set low_memory=False.
  chat = pd.read_csv('raw data/FinalCHAT_72909.csv')


In [5]:
chat_dict = pd.read_excel('raw data/chat_dictionary.xlsx', skipfooter=2)
chat_dict

,VARIABLE NAME,DEFINITION,SOURCE
0,ag_harvester,Number of self‐propelled machines that reap an...,FAOSTAT (2004)
1,ag_milkingmachine,Number of installations consisting of several ...,FAOSTAT (2004)
2,ag_tractor,Number of wheel and crawler tractors (excludin...,FAOSTAT (2004)
3,atm,Number of electromechanical devices that permi...,BIS (various years) and ECB (various years)
4,aviationpkm,Civil aviation passenger‐KM traveled on schedu...,Mitchell (1998)
...,...,...,...
106,pctivprimeenroll,Primary school enrollment rate,Mitchell and Banks
107,pctivsecenroll,Secondary school enrollment rate,Mitchell and Banks
108,pctivprivateinv,Private investment as a share of GDP,Penn World Table (2007)
109,pctivpublicinv,Public investment as a share of GDP,Penn World Table (2007)


In [6]:
var_names = list(chat.columns[2:])
var_names

['ag_harvester',
 'ag_milkingmachine',
 'ag_tractor',
 'atm',
 'aviationpkm',
 'aviationtkm',
 'bed_acute',
 'bed_hosp',
 'bed_longterm',
 'cabletv',
 'cellphone',
 'cheque',
 'computer',
 'creditdebit',
 'eft',
 'elecprod',
 'fert_total',
 'internetuser',
 'irrigatedarea',
 'kidney_dialpat',
 'kidney_homedialpat',
 'loom_auto',
 'loom_total',
 'mail',
 'med_catscanner',
 'med_lithotriptor',
 'med_mammograph',
 'med_mriunit',
 'med_radiationequip',
 'newspaper',
 'pctdaysurg_cataract',
 'pctdaysurg_cholecyst',
 'pctdaysurg_hernia',
 'pctdaysurg_lapcholecyst',
 'pctdaysurg_tonsil',
 'pctdaysurg_varicosevein',
 'pcthomedialysis',
 'pctimmunizdpt',
 'pctimmunizmeas',
 'pctirrigated',
 'pctmvbyarea',
 'pest_total',
 'pos',
 'radio',
 'railline',
 'railp',
 'railpkm',
 'railt',
 'railtkm',
 'ship_all',
 'ship_motor',
 'ship_sail',
 'ship_steam',
 'ship_steammotor',
 'shipton_all',
 'shipton_motor',
 'shipton_sail',
 'shipton_steam',
 'shipton_steammotor',
 'spindle_mule',
 'spindle_ring',
 

In [7]:
def read_chat(col_num, tech_name, unit='Number in Use', metric='Cumulative total capacity'):
    chat = pd.read_csv('raw data/FinalCHAT_72909.csv', usecols=[0,1,col_num], dtype={col_num:float},
                      na_values=' ')
    chat.replace('Venezuala', 'Venezuela', inplace=True)
    chat = chat[chat['country_name']!='Indochina']
    country_list = list(set(chat['country_name']))
    chat_df = []
    for country in country_list:
        df = chat[chat['country_name']==country]
        df.set_index('year', drop=True, inplace=True)
        df = df.transpose()
        df['Country Name'] = country
        if country == 'North Vietnam':
            country_code = 'VD'
        elif country == 'South Vietnam':
            country_code = 'VNM'
        elif country == 'Czechoslovakia':
            country_code = 'CSK'
        else:
            country_code = coco.convert(names=country, to='iso2')
        df['Country Code'] = country_code
        df.drop(index='country_name', inplace=True)
        chat_df.append(df)
    chat_all = pd.concat(chat_df)
    chat_all['Data Source'] = 'CHAT'
    chat_all['Spatial Scale'] = 'National'
    chat_all['Unit'] = unit
    chat_all['Metric'] = metric
    chat_all['Technology Name'] = tech_name
    chat_all['ID'] = chat_all['Technology Name'] + '_' + chat_all['Metric'] + '_' + chat_all['Country Code']
    chat_all.set_index('ID', drop=True, inplace=True)
    chat_all.columns.name = None
    omit = []
    for col in chat_all.columns:
        if type(col) != int:
            omit.append(col)
    empty_rows = chat_all.drop(columns=omit)
    empty_rows.dropna(how='all', inplace=True)
    na_idx = []
    for country in chat_all.index:
        if country not in empty_rows.index:
            na_idx.append(country)
    chat_all.drop(na_idx, inplace=True)
    file_name = 'cleaned data/chat_' + tech_name + '.csv'
    chat_all.to_csv(file_name)
    return chat_all


In [8]:
ag_harvester = read_chat(2, 'Crop Harvester', metric='Total Number')
ag_harvester

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Crop Harvester_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number in Use,Total Number,Crop Harvester
Crop Harvester_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number in Use,Total Number,Crop Harvester
Crop Harvester_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number in Use,Total Number,Crop Harvester
Crop Harvester_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number in Use,Total Number,Crop Harvester
Crop Harvester_Total Number_LS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lesotho,LS,CHAT,National,Number in Use,Total Number,Crop Harvester
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Crop Harvester_Total Number_NZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,New Zealand,NZ,CHAT,National,Number in Use,Total Number,Crop Harvester
Crop Harvester_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Italy,IT,CHAT,National,Number in Use,Total Number,Crop Harvester
Crop Harvester_Total Number_AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United Arab Emirates,AE,CHAT,National,Number in Use,Total Number,Crop Harvester


In [9]:
steam_ships = read_chat(54, 'Steam Ships')
steam_ships

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Steam Ships_Cumulative total capacity_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,11791.0,United Kingdom,GB,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_NL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Netherlands,NL,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_CL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Chile,CL,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1826.0,Norway,NO,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_BE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Belgium,BE,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,France,FR,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships
Steam Ships_Cumulative total capacity_CU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,38.0,Cuba,CU,CHAT,National,Number in Use,Cumulative total capacity,Steam Ships


In [10]:
telephones = read_chat(91, 'Telephones', metric='Total Number')
telephones

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Telephones_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number in Use,Total Number,Telephones
Telephones_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number in Use,Total Number,Telephones
Telephones_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number in Use,Total Number,Telephones
Telephones_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number in Use,Total Number,Telephones
Telephones_Total Number_LS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lesotho,LS,CHAT,National,Number in Use,Total Number,Telephones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Telephones_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Peru,PE,CHAT,National,Number in Use,Total Number,Telephones
Telephones_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,114000.0,Italy,IT,CHAT,National,Number in Use,Total Number,Telephones
Telephones_Total Number_AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United Arab Emirates,AE,CHAT,National,Number in Use,Total Number,Telephones


In [11]:
rail = read_chat(46, 'Railroad', unit='Length of railway open in kilometers',metric='Cumulative Length')
rail

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Railroad_Cumulative Length_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
Railroad_Cumulative Length_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1166.0,Colombia,CO,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
Railroad_Cumulative Length_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
Railroad_Cumulative Length_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
Railroad_Cumulative Length_TW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,647.0,Taiwan,TW,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Railroad_Cumulative Length_SO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,Somalia,SO,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
Railroad_Cumulative Length_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2116.0,Peru,PE,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad
Railroad_Cumulative Length_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,20304.0,Italy,IT,CHAT,National,Length of railway open in kilometers,Cumulative Length,Railroad


In [12]:
tv = read_chat(97, 'Television', metric='Total Number')
tv

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Television_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number in Use,Total Number,Television
Television_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number in Use,Total Number,Television
Television_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number in Use,Total Number,Television
Television_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number in Use,Total Number,Television
Television_Total Number_LS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lesotho,LS,CHAT,National,Number in Use,Total Number,Television
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Television_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Peru,PE,CHAT,National,Number in Use,Total Number,Television
Television_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Italy,IT,CHAT,National,Number in Use,Total Number,Television
Television_Total Number_AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United Arab Emirates,AE,CHAT,National,Number in Use,Total Number,Television


In [13]:
# Radio	CHAT database (radio)
radio = read_chat(45, 'Radio', metric='Total Number')
radio

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Radio_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number in Use,Total Number,Radio
Radio_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number in Use,Total Number,Radio
Radio_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number in Use,Total Number,Radio
Radio_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number in Use,Total Number,Radio
Radio_Total Number_LS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lesotho,LS,CHAT,National,Number in Use,Total Number,Radio
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Radio_Total Number_SO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Somalia,SO,CHAT,National,Number in Use,Total Number,Radio
Radio_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Peru,PE,CHAT,National,Number in Use,Total Number,Radio
Radio_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Italy,IT,CHAT,National,Number in Use,Total Number,Radio


In [14]:
# Passenger Cars	CHAT (vehicle_car)
cars = read_chat(102, 'Passenger Cars', metric='Total Number')
cars

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Passenger Cars_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number in Use,Total Number,Passenger Cars
Passenger Cars_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number in Use,Total Number,Passenger Cars
Passenger Cars_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number in Use,Total Number,Passenger Cars
Passenger Cars_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number in Use,Total Number,Passenger Cars
Passenger Cars_Total Number_LS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lesotho,LS,CHAT,National,Number in Use,Total Number,Passenger Cars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Passenger Cars_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Peru,PE,CHAT,National,Number in Use,Total Number,Passenger Cars
Passenger Cars_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,23900.0,Italy,IT,CHAT,National,Number in Use,Total Number,Passenger Cars
Passenger Cars_Total Number_AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United Arab Emirates,AE,CHAT,National,Number in Use,Total Number,Passenger Cars


In [15]:
# Telegraph Traffic	CHAT (telegram)

telegram = read_chat(90, 'Telegraph Traffic', 'Number of telegrams sent annually', 'Total Number')
telegram

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Telegraph Traffic_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
Telegraph Traffic_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
Telegraph Traffic_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
Telegraph Traffic_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
Telegraph Traffic_Total Number_TW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.964,Taiwan,TW,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Telegraph Traffic_Total Number_CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,10.0,Canada,CA,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
Telegraph Traffic_Total Number_NZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.704409,New Zealand,NZ,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic
Telegraph Traffic_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.9,Peru,PE,CHAT,National,Number of telegrams sent annually,Total Number,Telegraph Traffic


In [16]:
# Postal traffic	CHAT (mail)

mail = read_chat(25, 'Postal Traffic', 'Number of items mailed/received', metric='Total Number')
mail

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Postal Traffic_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
Postal Traffic_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
Postal Traffic_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
Postal Traffic_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
Postal Traffic_Total Number_TW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,54.0,Taiwan,TW,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Postal Traffic_Total Number_NZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,238.0,New Zealand,NZ,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
Postal Traffic_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,17.0,Peru,PE,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic
Postal Traffic_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2127.0,Italy,IT,CHAT,National,Number of items mailed/received,Total Number,Postal Traffic


In [17]:
# Cellphones	CHAT (cellphone)

cells = read_chat(12, 'Cellphones', unit='Number of users', metric='Total Number')
cells

/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Data Source'] = 'CHAT'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chat_all['Spatial Scale'] = 'National'
/var/folders/4_/pdkx43_96v7dxssy4jtd5vyc0000gn/T/ipykernel_13864/2545176736.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,...,2007,2008,1919,Country Name,Country Code,Data Source,Spatial Scale,Unit,Metric,Technology Name
ID,,,,,,,,,,,,,,,,,,,,,
Cellphones_Total Number_DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Germany,DE,CHAT,National,Number of users,Total Number,Cellphones
Cellphones_Total Number_CO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Colombia,CO,CHAT,National,Number of users,Total Number,Cellphones
Cellphones_Total Number_HU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Hungary,HU,CHAT,National,Number of users,Total Number,Cellphones
Cellphones_Total Number_ZM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Zambia,ZM,CHAT,National,Number of users,Total Number,Cellphones
Cellphones_Total Number_LS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Lesotho,LS,CHAT,National,Number of users,Total Number,Cellphones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Cellphones_Total Number_PE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Peru,PE,CHAT,National,Number of users,Total Number,Cellphones
Cellphones_Total Number_IT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Italy,IT,CHAT,National,Number of users,Total Number,Cellphones
Cellphones_Total Number_AE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,United Arab Emirates,AE,CHAT,National,Number of users,Total Number,Cellphones


In [18]:
for var_def in chat_dict[chat_dict[chat_dict.columns[0]]=='telephone'][chat_dict.columns[1]]:
    print(var_def)

Number of mainline telephone lines connecting a customer's equipment to the public switched telephone network as of year end


In [19]:
for var_idx in chat_dict[chat_dict[chat_dict.columns[0]]=='ag_harvester'].index:
    print(var_idx)

0


In [20]:
chat_dict.iloc[89]

VARIABLE NAME                                             telephone
DEFINITION        Number of mainline telephone lines connecting ...
SOURCE                           Mitchell (1998), WORLD BANK (2007)
Name: 89, dtype: object